# Preprocessing

In [1]:
import numpy as np
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import csv
np.random.seed(42)

from tqdm import tqdm

## Downsample and Filtering
Trim down the dataset to a tenth of the size so that the next processing steps can be done quicker.

In [2]:
# print(chunk.dtypes)
# s = """...""".split()
# for i in range(0, len(s), 2):
#     print(f'"{s[i]}": "{s[i+1]}",')

# Define a schema so that the datatypes are the same across the chunks
schema = {
    "recommendationid": "Int64",
    "appid": "Int64",
    "game": "object",
    "author_steamid": "Int64",
    "author_num_games_owned": "Int64",
    "author_num_reviews": "Int64",
    "author_playtime_forever": "Int64",
    "author_playtime_last_two_weeks": "Int64",
    "author_playtime_at_review": "Int64",
    "author_last_played": "Int64",
    "language": "object",
    "review": "object",
    "timestamp_created": "Int64",
    "timestamp_updated": "Int64",
    "voted_up": "Int64",
    "votes_up": "Int64",
    "votes_funny": "Int64",
    "weighted_vote_score": "float64",
    "comment_count": "Int64",
    "steam_purchase": "Int64",
    "received_for_free": "Int64",
    "written_during_early_access": "Int64",
    "hidden_in_steam_china": "Int64",
    "steam_china_location": "object",
}

In [4]:
# Trim down the dataset to a tenth to make next steps quicker
all_path = "steam_dataset/all_reviews.csv"
subset_path = "steam_dataset/subset_reviews.csv"

process = lambda x: x.sample(frac=0.1, random_state=np.random.randint(0, 100))
all_reader = pd.read_csv(all_path, chunksize=100_000, dtype=schema)
    
# First write will create/truncate file and write the header
chunk = process(next(all_reader))
chunk.to_csv(subset_path, mode='w', header=True, index=False, quoting=csv.QUOTE_NONNUMERIC)

# about 20 mins
for c in tqdm(all_reader):
    # next writes will append to file and not write the header
    chunk = process(c)
    chunk.to_csv(subset_path, mode='a', header=False, index=False, quoting=csv.QUOTE_NONNUMERIC)

1138it [18:55,  1.00it/s]


Filter out non-English reviews to create an English only dataset. This will make it easier to perform analysis later.

In [5]:
all_path = "steam_dataset/subset_reviews.csv"
eng_path = "steam_dataset/eng_reviews.csv"

process = lambda x: x[x["language"] == "english"]
all_reader = pd.read_csv(all_path, chunksize=100_000, dtype=schema)

# First write will create/truncate file and write the header
chunk = process(next(all_reader))
chunk.to_csv(eng_path, mode='w', header=True, index=False, quoting=csv.QUOTE_NONNUMERIC)

for c in tqdm(all_reader):
    # next writes will append to file and not write the header
    chunk = process(c)
    chunk.to_csv(eng_path, mode='a', header=False, index=False, quoting=csv.QUOTE_NONNUMERIC)

113it [02:36,  1.38s/it]


Trim down the dataset even more until we have around 2 million for the medium dataset and 20K for the small dataset

In [6]:
# get the number of lines using 'wc -l' in the terminal
# the previous cells should be deterministic due to our seed
# we also want our small set to be a subset of the medium set
n = 5155338
med_n = 2_000_000
sml_n = 20_000

med_prob = med_n / n
# sml_prob = sml_n / n
sml_prob = sml_n / med_n

In [7]:
all_path = "steam_dataset/eng_reviews.csv"
med_path = "steam_dataset/med_sample.csv"
sml_path = "steam_dataset/sml_sample.csv"

med_process = lambda x: x.sample(frac=med_prob, random_state=np.random.randint(0, 100))
sml_process = lambda x: x.sample(frac=sml_prob, random_state=np.random.randint(0, 100))
all_reader = pd.read_csv(all_path, chunksize=100_000, dtype=schema)

# First write will create/truncate file and write the header
med_chunk = med_process(next(all_reader))
sml_chunk = sml_process(med_chunk)

med_chunk.to_csv(med_path, mode='w', header=True, index=False, quoting=csv.QUOTE_NONNUMERIC)
sml_chunk.to_csv(sml_path, mode='w', header=True, index=False, quoting=csv.QUOTE_NONNUMERIC)

for chunk in tqdm(all_reader):
    # next writes will append to file and not write the header
    med_chunk = med_process(chunk)
    sml_chunk = sml_process(med_chunk)
    
    med_chunk.to_csv(med_path, mode='a', header=False, index=False, quoting=csv.QUOTE_NONNUMERIC)
    sml_chunk.to_csv(sml_path, mode='a', header=False, index=False, quoting=csv.QUOTE_NONNUMERIC)

51it [01:04,  1.26s/it]


# Convert to Parquet 
Sanity check for our new datasets and filter out some columns, then convert to parquet

In [4]:
sml_path = "steam_dataset/sml_sample.csv"
sml = pd.read_csv(sml_path)
sml = sml.drop(['hidden_in_steam_china', 'steam_china_location'], axis='columns', errors='ignore')

print(len(sml))
# sml.head()
# sml.columns
print(sml.dtypes)

sml.to_csv(sml_path, index=False, quoting=csv.QUOTE_NONNUMERIC)

20003
recommendationid                    int64
appid                               int64
game                               object
author_steamid                      int64
author_num_games_owned              int64
author_num_reviews                  int64
author_playtime_forever             int64
author_playtime_last_two_weeks      int64
author_playtime_at_review           int64
author_last_played                  int64
language                           object
review                             object
timestamp_created                   int64
timestamp_updated                   int64
voted_up                            int64
votes_up                            int64
votes_funny                         int64
weighted_vote_score               float64
comment_count                       int64
steam_purchase                      int64
received_for_free                   int64
written_during_early_access         int64
dtype: object


In [5]:
med_path = "steam_dataset/med_sample.csv"
med = pd.read_csv(med_path)
med = med.drop(['hidden_in_steam_china', 'steam_china_location'], axis='columns', errors='ignore')

print(len(med))
# med.head()
# med.columns
print(med.dtypes)

med.to_csv(med_path, index=False, quoting=csv.QUOTE_NONNUMERIC)

2000013
recommendationid                    int64
appid                               int64
game                               object
author_steamid                      int64
author_num_games_owned              int64
author_num_reviews                  int64
author_playtime_forever             int64
author_playtime_last_two_weeks      int64
author_playtime_at_review           int64
author_last_played                  int64
language                           object
review                             object
timestamp_created                   int64
timestamp_updated                   int64
voted_up                            int64
votes_up                            int64
votes_funny                         int64
weighted_vote_score               float64
comment_count                       int64
steam_purchase                      int64
received_for_free                   int64
written_during_early_access         int64
dtype: object


In [6]:
med_path_parquet = 'steam_dataset/med_sample.parquet'
sml_path_parquet = 'steam_dataset/sml_sample.parquet'

table = pa.Table.from_pandas(pd.read_csv(med_path))
pq.write_table(table, med_path_parquet)

table = pa.Table.from_pandas(pd.read_csv(sml_path))
pq.write_table(table, sml_path_parquet)